Recorded ping tests: 1 per minute, single ping, to 8.8.8.8 (google dns)

Data quality notes: At one point in time we had a timezone jump because the timezone was corrected on the measuring device. Timezone was recorded, but I chose to keep processing simple and ignore timezones as much as possible

In [17]:
# Input file: contains output of 'date' and 'ping'
!head -n 20 googledns.log

--- 8.8.8.8 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 17.268/17.268/17.268/0.000 ms
Sat 01 Aug 2020 12:00:01 AM BST
PING 8.8.8.8 (8.8.8.8) 56(84) bytes of data.
64 bytes from 8.8.8.8: icmp_seq=1 ttl=117 time=51.9 ms

--- 8.8.8.8 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 51.918/51.918/51.918/0.000 ms
Sat 01 Aug 2020 12:01:01 AM BST
PING 8.8.8.8 (8.8.8.8) 56(84) bytes of data.
64 bytes from 8.8.8.8: icmp_seq=1 ttl=117 time=19.2 ms

--- 8.8.8.8 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 19.151/19.151/19.151/0.000 ms
Sat 01 Aug 2020 12:02:02 AM BST
PING 8.8.8.8 (8.8.8.8) 56(84) bytes of data.
64 bytes from 8.8.8.8: icmp_seq=1 ttl=117 time=20.1 ms


In [29]:
import re
from itertools import tee, filterfalse

In [30]:
lines = open('googledns.log').readlines()
lines = [l.strip() for l in lines]
datepattern = re.compile('.*2020.*')
dates = list(filter(lambda line: datepattern.match(line), lines))
print("First and last timestamp:")
print(dates[0])
print(dates[-1])
print(len(dates))

# here you can see that TZ change mentioned at the top

First and last timestamp:
Sat 01 Aug 2020 12:00:01 AM BST
Sun 09 Aug 2020 08:59:01 PM CEST
12719


In [31]:
resultpattern = re.compile('1 packets transmitted')
results = list(filter(lambda line: resultpattern.match(line), lines))
print("Expected # ping tests:", 60*24*8 + 60*21) # about 8 full days + 21 hours on the last date
print("Actual # ping tests:", len(results))

# looks good - again, off by 60 because of the TZ change

Expected # ping tests: 12780
Actual # ping tests: 12719


In [32]:
# Now let's check successful & failed pings
def is_good(result):
    return '1 received' in result
def is_bad(result):
    return '0 received' in result
    
success_count = len(list(filter(is_good,results)))
fail_count = len(list(filter(is_bad,results)))
print("Successful:", success_count)
print("Failures  :", fail_count)
print("Total     :", (success_count + fail_count))

Successful: 11060
Failures  : 1659
Total     : 12719


In [33]:
# How many hours of failure is that?
print("Estimated hour of failure:", (fail_count / 60))

Estimated hour of failure: 27.65


In [49]:
# Let's find when things went wrong
combined = list(zip(dates,results))

# find points at which is_good(previous) XOR is_good(current)
changepoints = [(0, dates[0], results[0])]
for i, result in enumerate(results[:-1]):
    if is_good(result) ^ is_good(results[i+1]):
        changepoints.append((i+1, dates[i+1], results[i+1]))

In [50]:
changepoints

[(0,
  'Sat 01 Aug 2020 12:00:01 AM BST',
  '1 packets transmitted, 1 received, 0% packet loss, time 0ms'),
 (303,
  'Sat 01 Aug 2020 05:03:01 AM BST',
  '1 packets transmitted, 0 received, 100% packet loss, time 0ms'),
 (700,
  'Sat 01 Aug 2020 11:41:01 AM BST',
  '1 packets transmitted, 1 received, 0% packet loss, time 0ms'),
 (898,
  'Sat 01 Aug 2020 02:59:01 PM BST',
  '1 packets transmitted, 0 received, 100% packet loss, time 0ms'),
 (912,
  'Sat 01 Aug 2020 03:13:01 PM BST',
  '1 packets transmitted, 1 received, 0% packet loss, time 0ms'),
 (9440,
  'Fri 07 Aug 2020 01:21:01 PM BST',
  '1 packets transmitted, 0 received, 100% packet loss, time 0ms'),
 (9445,
  'Fri 07 Aug 2020 01:26:01 PM BST',
  '1 packets transmitted, 1 received, 0% packet loss, time 0ms'),
 (9457,
  'Fri 07 Aug 2020 01:38:01 PM BST',
  '1 packets transmitted, 0 received, 100% packet loss, time 0ms'),
 (9459,
  'Fri 07 Aug 2020 01:40:02 PM BST',
  '1 packets transmitted, 1 received, 0% packet loss, time 0ms'),


In [60]:
for i in range(len(changepoints)):
    offset, date, result = changepoints[i]
    if is_bad(result):
        enddate = changepoints[i+1][1]
        minutes = changepoints[i+1][0] - offset
        print("Outage from %s to %s (approx %2.2f hours)"%(date, enddate, minutes*1.0/60))

Outage from Sat 01 Aug 2020 05:03:01 AM BST to Sat 01 Aug 2020 11:41:01 AM BST (approx 6.62 hours)
Outage from Sat 01 Aug 2020 02:59:01 PM BST to Sat 01 Aug 2020 03:13:01 PM BST (approx 0.23 hours)
Outage from Fri 07 Aug 2020 01:21:01 PM BST to Fri 07 Aug 2020 01:26:01 PM BST (approx 0.08 hours)
Outage from Fri 07 Aug 2020 01:38:01 PM BST to Fri 07 Aug 2020 01:40:02 PM BST (approx 0.03 hours)
Outage from Fri 07 Aug 2020 02:04:01 PM BST to Fri 07 Aug 2020 08:20:01 PM BST (approx 6.27 hours)
Outage from Fri 07 Aug 2020 09:33:01 PM CEST to Fri 07 Aug 2020 09:38:01 PM CEST (approx 0.08 hours)
Outage from Sat 08 Aug 2020 05:04:01 AM CEST to Sat 08 Aug 2020 05:06:01 AM CEST (approx 0.03 hours)
Outage from Sat 08 Aug 2020 01:44:01 PM CEST to Sat 08 Aug 2020 09:31:01 PM CEST (approx 7.78 hours)
Outage from Sat 08 Aug 2020 09:32:01 PM CEST to Sat 08 Aug 2020 09:35:01 PM CEST (approx 0.05 hours)
Outage from Sat 08 Aug 2020 09:36:02 PM CEST to Sat 08 Aug 2020 09:40:01 PM CEST (approx 0.07 hours)
